### UK CONFECTIONAERY DATA ANALYSIS

#### IMPORTING THE REQUIRED LIBRARIES

In [169]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import KNNImputer

### DATA IMPORTATION

In [170]:
data = pd.read_excel("Data set_confectionary_4010.xlsx", header=0)


In [171]:
conf_data = data.copy(deep=True)
conf_data.head(5)

,Date,Country(UK),Confectionary,Units Sold,Revenue(£),Cost(£),Profit(£)
0,2002-11-11,England,Biscuit,1118.0,5590.0,2459.6,3130.4
1,2002-07-05,England,Biscuit,708.0,3540.0,1557.6,1982.4
2,2001-10-31,England,Biscuit,1269.0,6345.0,2791.8,3553.2
3,2004-09-13,England,Biscuit,1631.0,8155.0,3588.2,4566.8
4,2004-03-10,England,Biscuit,2240.0,11200.0,4928.0,6272.0


### DATA EXPLORATION

In [172]:
rows, columns = conf_data.shape
print(f"Rows: {rows}, Columns: {columns}")

Rows: 1001, Columns: 7


### Data column names

In [173]:
conf_data.columns

Index(['Date', 'Country(UK)', 'Confectionary', 'Units Sold', 'Revenue(£)',
       'Cost(£)', 'Profit(£)'],
      dtype='object')

### Rename columns. remove POUNDS sign, replace space with _ etc

In [174]:
new_columns = []
for column in conf_data.columns:
    new_col_name = column.replace('£', '').replace('(', '').replace(')', '').replace('uk','').replace(' ','_').lower().strip()
    new_columns.append(new_col_name)

conf_data.columns = new_columns
conf_data.rename(columns={'countryuk': 'country'}, inplace=True)
conf_data.columns

Index(['date', 'country', 'confectionary', 'units_sold', 'revenue', 'cost',
       'profit'],
      dtype='object')

### Checking the data types

In [175]:
dtypes_df = pd.DataFrame({
    'column': conf_data.columns,
    'data_type': conf_data.dtypes
    })
dtypes_df.reset_index(inplace=True, drop=True)
dtypes_df

,column,data_type
0,date,datetime64[ns]
1,country,object
2,confectionary,object
3,units_sold,float64
4,revenue,float64
5,cost,float64
6,profit,float64


## Exploring column data

In [176]:
unique_values_df = pd.DataFrame({
    'column': conf_data.columns,
    'unique_values': conf_data.nunique()
})
unique_values_df.reset_index(inplace=True, drop=True)
unique_values_df

,column,unique_values
0,date,802
1,country,5
2,confectionary,8
3,units_sold,391
4,revenue,501
5,cost,513
6,profit,511


In [177]:
conf_data.country.unique()

array(['England', 'Scotland', 'Wales', 'N. Ireland', 'Jersey'],
      dtype=object)

### Checking for missing values

### Rows without any data

In [178]:
nan_rows_count = conf_data.isna().all(axis=1).sum()
print("Rows with no data: ",int(nan_rows_count))

Rows with no data:  0


In [179]:
nan_per_column = pd.DataFrame({
    'column': conf_data.columns,
    'missing_data': conf_data.isna().sum(),
    'percentages_of_missing_values': (conf_data.isna().sum()/len(conf_data)) * 100
})
nan_per_column.reset_index(inplace=True, drop=True)
nan_per_column

,column,missing_data,percentages_of_missing_values
0,date,0,0.000000
1,country,0,0.000000
2,confectionary,0,0.000000
3,units_sold,5,0.499500
4,revenue,9,0.899101
5,cost,9,0.899101
6,profit,3,0.299700


In [180]:
conf_data.describe().T

,count,mean,min,25%,50%,75%,max,std
date,1001,2002-11-19 10:54:32.727272832,2000-01-02 00:00:00,2001-05-12 00:00:00,2002-11-10 00:00:00,2004-04-25 00:00:00,2005-12-28 00:00:00,NaN
units_sold,996.0,1633.360442,200.0,923.0,1530.5,2300.0,4493.0,876.356045
revenue,992.0,6847.601815,200.0,2993.0,5967.0,9505.0,23988.0,4699.184062
cost,992.0,2820.190877,40.0,1204.0,2456.8,3977.625,10994.5,2073.969135
profit,998.0,4012.076052,160.0,1872.4,3459.0,5445.0,13479.0,2648.166312


## REPLACEMENT OF NaN

In [181]:
conf_data_without_nan = conf_data.dropna()
rows_with_nan_data = len(conf_data) - len(conf_data_without_nan)
print("Number of rows with NaN values is: ",rows_with_nan_data)
print("Percentage of rows with NaN values is: ",(rows_with_nan_data/len(conf_data_without_nan))*100)

Number of rows with NaN values is:  25
Percentage of rows with NaN values is:  2.5614754098360657


### ANALYSIS


In [182]:
# conf_data['day_name'] = conf_data['date'].dt.day_name()
# # Classify as Weekend or Weekday
# conf_data['weekend'] = np.where(conf_data['day_name'].isin(['Saturday', 'Sunday']), 'Weekend', 'Weekday')
# conf_data.head()


### REPLACE NaN

## USE KNN METHOD
### Encode categorical data in columns country and confectionary

In [183]:
conf_data.country.unique()

array(['England', 'Scotland', 'Wales', 'N. Ireland', 'Jersey'],
      dtype=object)

In [184]:
conf_data.confectionary.unique()

array(['Biscuit', 'Biscuit Nut', 'Choclate Chunk', 'Caramel nut',
       'Caramel', 'Plain', 'Chocolate Chunk', 'Caramel Nut'], dtype=object)

In [185]:
# Define mappings
country_map = {'England': 1, 'Scotland': 2, 'Wales': 3, 'N. Ireland': 4, 'Jersey': 5}
confectionary_map = {'Biscuit':1, 'Biscuit Nut': 2, 'Choclate Chunk': 3, 'Caramel nut':4,
       'Caramel':5, 'Plain':6, 'Chocolate Chunk':7, 'Caramel Nut':8}

# Apply mappings
conf_data['country'] = conf_data['country'].map(country_map)
conf_data['confectionary'] = conf_data['confectionary'].map(confectionary_map)
print(conf_data.country.unique())
conf_data.confectionary.unique()

[1 2 3 4 5]


array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [186]:
conf_data.head()

,date,country,confectionary,units_sold,revenue,cost,profit
0,2002-11-11,1,1,1118.0,5590.0,2459.6,3130.4
1,2002-07-05,1,1,708.0,3540.0,1557.6,1982.4
2,2001-10-31,1,1,1269.0,6345.0,2791.8,3553.2
3,2004-09-13,1,1,1631.0,8155.0,3588.2,4566.8
4,2004-03-10,1,1,2240.0,11200.0,4928.0,6272.0


### drop date column 

In [187]:
knn_data = conf_data.drop(columns=['date'], axis=1)
knn_data.head()

,country,confectionary,units_sold,revenue,cost,profit
0,1,1,1118.0,5590.0,2459.6,3130.4
1,1,1,708.0,3540.0,1557.6,1982.4
2,1,1,1269.0,6345.0,2791.8,3553.2
3,1,1,1631.0,8155.0,3588.2,4566.8
4,1,1,2240.0,11200.0,4928.0,6272.0


### Seperate categorical and numerical columns

In [188]:
numerical_cols = ['units_sold', 'revenue', 'cost', 'profit']   
categorical_cols = ['country', 'confectionary']    

In [189]:
# Scale only numeric columns
scaler = MinMaxScaler()
#scaler = StandardScaler()
scaled_numeric_data = scaler.fit_transform(knn_data[numerical_cols])

# Keep categorical columns as they are
categorical_data = knn_data[categorical_cols].values

full_data = np.hstack((scaled_numeric_data, categorical_data))

### impute the missing values

In [190]:
# Apply KNN imputer on full data
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(full_data)

column_names = ['units_sold', 'revenue', 'cost', 'profit','country', 'confectionary'] # ALWAYS START WITH NUMERICAL,THEN CATEGORICAL
imputed_scaled_df = pd.DataFrame(imputed_data, columns= column_names)
imputed_scaled_df.head()

,units_sold,revenue,cost,profit,country,confectionary
0,0.213836,0.226585,0.220877,0.223020,1.0,1.0
1,0.118332,0.140407,0.138537,0.136827,1.0,1.0
2,0.249010,0.258324,0.251203,0.254764,1.0,1.0
3,0.333333,0.334412,0.323903,0.330866,1.0,1.0
4,0.475192,0.462418,0.446209,0.458893,1.0,1.0


### Confirm all NaN values are filled out

In [191]:
nan_per_column = pd.DataFrame({
    'column': imputed_scaled_df.columns,
    'missing_data': imputed_scaled_df.isna().sum(),
    'percentages_of_missing_values': (imputed_scaled_df.isna().sum()/len(imputed_scaled_df)) * 100
})
nan_per_column.reset_index(inplace=True, drop=True)
nan_per_column

,column,missing_data,percentages_of_missing_values
0,units_sold,0,0.0
1,revenue,0,0.0
2,cost,0,0.0
3,profit,0,0.0
4,country,0,0.0
5,confectionary,0,0.0


### Inverse transform and combining with categorical data

In [192]:
# Inverse transform to get back original scale
imputed_df = pd.DataFrame(scaler.inverse_transform(imputed_scaled_df[numerical_cols]), columns=numerical_cols)
imputed_scaled_df.update(imputed_df)
imputed_scaled_df['date'] = conf_data['date']
conf_data = imputed_scaled_df
conf_data['country'] = conf_data['country'].astype(int)
conf_data['confectionary'] = conf_data['confectionary'].astype(int)
conf_data.head()


,units_sold,revenue,cost,profit,country,confectionary,date
0,1118.0,5590.0,2459.6,3130.4,1,1,2002-11-11
1,708.0,3540.0,1557.6,1982.4,1,1,2002-07-05
2,1269.0,6345.0,2791.8,3553.2,1,1,2001-10-31
3,1631.0,8155.0,3588.2,4566.8,1,1,2004-09-13
4,2240.0,11200.0,4928.0,6272.0,1,1,2004-03-10


### Confirming the data types and shape after imputing

In [193]:
rows, columns = conf_data.shape
print(f"Rows: {rows}, Columns: {columns}")

Rows: 1001, Columns: 7


In [194]:
dtypes_df = pd.DataFrame({
    'column': conf_data.columns,
    'data_type': conf_data.dtypes
    })
dtypes_df.reset_index(inplace=True, drop=True)
dtypes_df

,column,data_type
0,units_sold,float64
1,revenue,float64
2,cost,float64
3,profit,float64
4,country,int32
5,confectionary,int32
6,date,datetime64[ns]


In [195]:
imputed_scaled_df['date'] = conf_data['date']
conf_data = imputed_scaled_df
conf_data = conf_data[['date', 'country', 'confectionary', 'units_sold', 'revenue', 'cost', 'profit']]      
conf_data.head()

,date,country,confectionary,units_sold,revenue,cost,profit
0,2002-11-11,1,1,1118.0,5590.0,2459.6,3130.4
1,2002-07-05,1,1,708.0,3540.0,1557.6,1982.4
2,2001-10-31,1,1,1269.0,6345.0,2791.8,3553.2
3,2004-09-13,1,1,1631.0,8155.0,3588.2,4566.8
4,2004-03-10,1,1,2240.0,11200.0,4928.0,6272.0
